In [ ]:
import os

os.environ["ROBOFLOW_API_KEY"] = "Uj5MjQA12mDv8vICzSaL"

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key = "Uj5MjQA12mDv8vICzSaL")
project = rf.workspace("riz-yak11").project("ultimate-sykrp")
version = project.version(1)
dataset = version.download("yolov12")

In [ ]:
!ls {dataset.location}

In [ ]:
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

In [ ]:
!cat {dataset.location}/data.yaml

In [ ]:
from ultralytics import YOLO
import supervision as sv

model = YOLO(f'yolov12nultimatekaggle.pt')

# First, let's fix the data.yaml file
import yaml

# Read and fix the data.yaml file properly
data_yaml_path = f"{dataset.location}/data.yaml"

# Create a properly formatted data.yaml file
data_content = {
    'nc': 2,  # number of classes
    'names': ['0', '1'],  # class names
    'test': '../test/images',
    'train': '../train/images', 
    'val': '../valid/images'
}

# Write the corrected data.yaml
with open(data_yaml_path, 'w') as file:
    yaml.dump(data_content, file, default_flow_style=False)

# Now create the dataset
ds = sv.DetectionDataset.from_yolo(
    images_directory_path = f"{dataset.location}/test/images",
    annotations_directory_path = f"{dataset.location}/test/labels",
    data_yaml_path = f"{dataset.location}/data.yaml"
)

ds.classes

In [ ]:
import random

i = random.randint(0, len(ds))

images_path, image, labels = ds[i]

results = model(image, verbose = False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLO

image_path = f"nobas.jpg"
image = cv2.imread(image_path)

model = YOLO(f'yolov12nultimatekaggle.pt')

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

In [ ]:
from supervision.metrics import MeanAveragePrecision
from ultralytics import YOLO

model = YOLO(f'yolov12nultimatekaggle.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose = False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

In [ ]:
print("mAP 50:95 :", map.map50_95)
print("mAP 50 :", map.map50)
print("mAP 75 :", map.map75)

In [ ]:
map.plot()

In [ ]:
# import cv2
# import supervision as sv
# from ultralytics import YOLO

# model = YOLO('yolov12nultimatekaggle.pt')

# cap = cv2.VideoCapture(0)

# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# while True:
#     ret, frame = cap.read()

#     if not ret:
#         break

#     results = model(frame, verbose=False)[0]
#     detections = sv.Detections.from_ultralytics(results)

#     # Check for "no-bus" detections and print "1"
#     for class_id in detections.class_id:
#         if ds.classes[class_id] == "1":  # assuming "1" is the "no-bus" class
#             print("1")

#     annotated_frame = frame.copy()
#     annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
#     annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

#     cv2.imshow('Real-time Detection', annotated_frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close the window
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# import cv2
# import supervision as sv
# from ultralytics import YOLO

# model = YOLO('yolov12nultimatekaggle.pt')

# cap = cv2.VideoCapture(0)

# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# while True:
#     ret, frame = cap.read()

#     if not ret:
#         break

#     results = model(frame, verbose=False)[0]
#     detections = sv.Detections.from_ultralytics(results)

#     # Check for "no-bus" detections and print "1"
#     for class_id in detections.class_id:
#         if ds.classes[class_id] == "1":  # assuming "1" is the "no-bus" class
#             print("1")

#     annotated_frame = frame.copy()
#     annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
#     annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

#     cv2.imshow('Real-time Detection', annotated_frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close the window
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLO

model = YOLO('yolov12nultimatekaggle.pt')

# Use video file instead of webcam (0)
video_path = "toll3.mp4"  # or "toll2.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError(f"Cannot open video file: {video_path}")

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Check for "no-bus" detections and print "1"
    for class_id in detections.class_id:
        if ds.classes[class_id] == "1":  # assuming "1" is the "no-bus" class
            print("Sensor nyala")

    annotated_frame = frame.copy()
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

    cv2.imshow('Real-time Detection', annotated_frame)

    # Optional: Add a small delay to control playback speed
    # cv2.waitKey(30) for ~30fps playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()

In [ ]:
# import cv2
# import supervision as sv
# from ultralytics import YOLO

# model = YOLO('yolov12nultimatekaggle.pt')

# # Use video file instead of webcam (0)
# video_path = "toll3.mp4"  # or "toll2.mp4"
# cap = cv2.VideoCapture(video_path)

# if not cap.isOpened():
#     raise IOError(f"Cannot open video file: {video_path}")

# # Get video properties
# fps = int(cap.get(cv2.CAP_PROP_FPS))
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# # Define the codec and create VideoWriter object
# output_path = "output_with_detections.mp4"
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can also use 'XVID'
# out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# frame_count = 0
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# while True:
#     ret, frame = cap.read()

#     if not ret:
#         break

#     results = model(frame, verbose=False)[0]
#     detections = sv.Detections.from_ultralytics(results)

#     # Check for "no-bus" detections and print "1"
#     for class_id in detections.class_id:
#         if ds.classes[class_id] == "1":  # assuming "1" is the "no-bus" class
#             print("Sensor nyala")

#     annotated_frame = frame.copy()
#     annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
#     annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections)

#     # Write the frame to output video
#     out.write(annotated_frame)

#     # Optional: Display the frame (you can comment this out if you don't want to see it)
#     cv2.imshow('Real-time Detection', annotated_frame)

#     # Progress tracking
#     frame_count += 1
#     if frame_count % 30 == 0:  # Print progress every 30 frames
#         print(f"Processed {frame_count}/{total_frames} frames")

#     # Optional: Add a small delay to control playback speed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release everything
# cap.release()
# out.release()  # Don't forget to release the output video writer
# cv2.destroyAllWindows()

# print(f"Output video saved as: {output_path}")